# Начало анализа

In [214]:
import pandas as pd
from currency_converter import CurrencyConverter



df = pd.read_csv('_data.csv', index_col=0)
# df.columns
# df['ID  объявления'].value_counts()
# df.head(30)
df.info()
# df['Количество комнат'].value_counts()
# df.loc[:15, 'Площадь, м2']

<class 'pandas.core.frame.DataFrame'>
Index: 23368 entries, 0 to 23367
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID  объявления            23368 non-null  int64  
 1   Количество комнат         22327 non-null  object 
 2   Тип                       23368 non-null  object 
 3   Метро                     22053 non-null  object 
 4   Адрес                     23368 non-null  object 
 5   Площадь, м2               23368 non-null  object 
 6   Дом                       23368 non-null  object 
 7   Парковка                  9951 non-null   object 
 8   Цена                      23368 non-null  object 
 9   Телефоны                  23368 non-null  object 
 10  Описание                  23368 non-null  object 
 11  Ремонт                    20613 non-null  object 
 12  Площадь комнат, м2        14458 non-null  object 
 13  Балкон                    15390 non-null  object 
 14  Окна       

### Достаем данные Москвы

In [215]:
# Город
df['city'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Адрес']]
df['city'].value_counts()
# df_msk = df[df['Адрес'].str.contains("Москва")].reset_index(drop = True)
df = df[df['city']=='Москва']
df.shape

(19737, 25)

## Работаем с пропусками

In [216]:
# Cмотрим NaN значения
df.isna().sum()

ID  объявления                  0
Количество комнат             535
Тип                             0
Метро                         346
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    11174
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2463
Площадь комнат, м2           7228
Балкон                       6630
Окна                         5150
Санузел                      2041
Можно с детьми/животными     4915
Дополнительно                 272
Название ЖК                 15281
Серия дома                  17646
Высота потолков, м           9202
Лифт                         4192
Мусоропровод                 8007
Ссылка на объявление            0
city                            0
dtype: int64

### Комнаты

In [217]:
# Получаем количество комнат
df.loc[:,'Rooms'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Количество комнат']]
df['Rooms'].unique()
df['Rooms'].value_counts()
df['Rooms'].isna().sum()

# Анализ выборки NaN в графе комнат показал, что это квартиры-студии, заменим NaN на 0
df.loc[df['Rooms'].isna()]['Описание'].head()
df['Rooms'].fillna(0, inplace=True)



### Метро

In [218]:
# Колонка метро
df['Subway'] = [row.split(' (')[0] if isinstance(row, str) else row for row in df['Метро']]
df['Subway'] = [row.split('. ')[1] if isinstance(row, str) else row for row in df['Subway']]
df = df.dropna(subset=['Метро'])

### Адрес

In [219]:
# Создаем переменную "за мкадом"
df['Адрес'].unique()
df['Not_Moscow'] = df['Адрес'].apply(lambda x: 1 if 'мкад' in str(x).lower() else 0)
df['Not_Moscow'].value_counts()

Not_Moscow
0    17972
1     1419
Name: count, dtype: int64

In [220]:
# Удаляем концы адреса по типу "ш. Варшавское (4 км до МКАД)"
df['New_address'] = [row.split('ш. ')[0] for row in df['Адрес']]
# df_msk = df_msk[df_msk['Адрес'] != 'Москва, Жилой Поколение кв-л'].reset_index(drop = True)

In [221]:
# время до метро пешком
df['time_to_subway'] = df['Метро'].str.extract('(\d+) мин пешком', expand=False)
df['time_to_subway'] = [row if isinstance(row, str) else 0 for row in df['time_to_subway']]
df['time_to_subway'] = pd.to_numeric(df['time_to_subway'])
# время до метро на машине
df['time_to_subway_car'] = df['Метро'].str.extract('(\d+) мин на машине', expand=False)
df['time_to_subway_car'] = [row if isinstance(row, str) else 0 for row in df['time_to_subway_car']]
df['time_to_subway_car'] = pd.to_numeric(df['time_to_subway_car'])
# конвертация времени на машине в время пешком
df['time_to_subway_car'] = 7 * df['time_to_subway_car']
df['time_to_subway_total'] = df['time_to_subway'] + df['time_to_subway_car']


### Площадь

In [222]:
df['Площадь, м2'].unique()
# Получим общую площадь квартиры
df['total_square'] = [row.split('.')[0] if isinstance(row, str) else row for row in df['Площадь, м2']]
df['total_square'].astype(int)
# df['total_square'].unique()

0        200
1        198
2        200
3        170
4         58
        ... 
23363     35
23364     38
23365     43
23366     52
23367     90
Name: total_square, Length: 19391, dtype: int64

In [223]:
# получим площадь кухни
df['Площадь, м2'].unique()
df['kitchen_square'] = [row.split('/')[-1] if isinstance(row, str) else row for row in df['Площадь, м2']]


### Животные и дети

In [224]:
# добавляем колонки /можно с детьми /можно с животными
df['Можно с детьми/животными'].unique()
df['ok_pets'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'животными' in str(x).lower() else 0)
df['ok_kids'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'детьми' in str(x).lower() else 0)


### Балкон

In [225]:
df['Балкон'].unique()

df['Balconies'] = df['Балкон'].str.extract(r'Балкон \((\d+)\)').astype(float)
df['Loggies'] = df['Балкон'].str.extract(r'Лоджия \((\d+)\)').astype(float)

df['Balconies'].isna().sum()
df['Balconies'] = df['Balconies'].fillna(df['Balconies'].mean())
# df_msk['Число балконов'].isna().sum()
df['Loggies'].isna().sum()
df['Loggies'] = df['Loggies'].fillna(df['Loggies'].mean())

### Санузел

In [226]:
 #делим сан узлы на количество совмещенных и раздельных
df['Санузел']
df['Connected_bathroom'] = df['Санузел'].str.extract(r'Совмещенный \((\d+)\)').astype(float)
df['Split_bathroom'] = df['Санузел'].str.extract(r'Раздельный \((\d+)\)').astype(float)

df['Connected_bathroom'] = df['Connected_bathroom'].fillna(df['Connected_bathroom'].mean())
df['Split_bathroom'] = df['Split_bathroom'].fillna(df['Split_bathroom'].mean())


### Мебель и кондиционер

In [227]:
df['Дополнительно']
# Мебель в комнатах
# Кондиционер
df['Furniture'] = df['Дополнительно'].str.extract('(ебель в комнатах)',).astype(str)
df['AC'] = df['Дополнительно'].str.extract('(ондиционер)',).astype(str)

df['Furniture'] = [1 if 'ебель в комнатах' in row else 0 for row in df['Furniture']]
df['AC'] = [1 if 'ондиционер' in row else 0 for row in df['AC']]


### Парковка

In [228]:
#делим парковку на платную и бесплатную
df['Парковка'] = df['Парковка'].replace(to_replace ='открытая', value = 'бесплатная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='наземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='многоуровневая', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='подземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='на крыше', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].fillna('бесплатная во дворе')
df['Parking'] = df['Парковка']
df = df.drop(['Парковка'], axis = 1)

### Дом (тип, этажность, этаж)

In [229]:
# разбитие колонки дом на тип, этажность, этаж квартиры
df['house_type'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['floors_amount'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['flat_floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

### Цена


In [230]:
c = CurrencyConverter()
def sale(text):
    result = None
    if text.split(' ')[1] == 'руб./':
        result = float(text.split(' ')[0])
    elif text.split(' ')[1] == '$,':
        result = c.convert(float(text.split(' ')[0]), 'USD', 'RUB')
    elif text.split(' ')[1] == '€,':
        result = c.convert(float(text.split(' ')[0]), 'EUR', 'RUB')
    return result
text = df['Цена']
text = text.apply(sale)
df['Price'] = text

In [231]:
# влючена ли комуналка
df['Utilities'] = df['Цена'].str.contains('Коммунальные услуги включены')
df['Rent_length'] = df['Цена'].str.contains('Срок аренды - Длительный')
df['Prepay'] = df['Цена'].str.contains('Предоплата')

In [232]:
# лифты
#заполняем нан по госту
df['Лифт'] = [
    row if row == row else
    ['Пасс (0), Груз (0)' if 5 >= df['floors_amount'].iloc[i]  else
     'Пасс (1), Груз (0)' if 5 <= df['floors_amount'].iloc[i] <= 9  else
     'Пасс (1), Груз (1)' if 10 <= df['floors_amount'].iloc[i] <= 19 else
     'Пасс (2), Груз (1)'][0]
    for i, row in enumerate(df['Лифт'])
]
#создаем отдельные колонки для количества пассажирских и грузовых лифтов
df['Pass_elevator'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Пасс' else '(0)' for row in df['Лифт']]
df['Cargo_elevator'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Груз' else '(0)' for row in df['Лифт']]

df['Pass_elevator'] = [row.split('(')[1] for row in df['Pass_elevator']]
df['Pass_elevator'] = [int(row.split(')')[0]) for row in df['Pass_elevator']]
df['Cargo_elevator'] = [row.split('(')[1] for row in df['Cargo_elevator']]
df['Cargo_elevator'] = [int(row.split(')')[0]) for row in df['Cargo_elevator']]
df = df.drop(['Лифт'], axis = 1)

### Окна, потолки, мусоропровод, ремонт

In [233]:
# очистка колонки окна
df['windows'] = df["Окна"].fillna('Во двор')

In [234]:
# заменяем нан в потолках на среднюю высоту
df['ceiling_height'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))

In [235]:
# заменяем нан на остутствие мусоропровода
df['Garbage_disposal'] = df['Мусоропровод'].fillna('Нет')

In [236]:
# учитываем нан как квартиры без ремонта
df['maintenance'] = df['Ремонт'].fillna('Без ремонта')

### Удаление старых / лишних колонок

In [237]:
# удаление старых колонок
df.drop([ 'Площадь, м2', 'Количество комнат', 'Тип', 'Адрес', 'Цена', 'Телефоны', 'Описание',
'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Мусоропровод',
'Ссылка на объявление', 'city', 'Метро'], axis=1, inplace=True)

In [238]:
df.dtypes

ID  объявления            int64
Rooms                    object
Subway                   object
Not_Moscow                int64
New_address              object
time_to_subway            int64
time_to_subway_car        int64
time_to_subway_total      int64
total_square             object
kitchen_square           object
ok_pets                   int64
ok_kids                   int64
Balconies               float64
Loggies                 float64
Connected_bathroom      float64
Split_bathroom          float64
Furniture                 int64
AC                        int64
Parking                  object
house_type               object
floors_amount             int64
flat_floor                int64
Price                   float64
Utilities                  bool
Rent_length                bool
Prepay                     bool
Pass_elevator             int64
Cargo_elevator            int64
windows                  object
ceiling_height          float64
Garbage_disposal         object
maintena

In [239]:
df = df.dropna(subset=['Price'])#так как в колонке прайс отсутсвуют некоторые значения мы удаляем объявления без цены

In [240]:
df.isna().sum()

ID  объявления          0
Rooms                   0
Subway                  0
Not_Moscow              0
New_address             0
time_to_subway          0
time_to_subway_car      0
time_to_subway_total    0
total_square            0
kitchen_square          0
ok_pets                 0
ok_kids                 0
Balconies               0
Loggies                 0
Connected_bathroom      0
Split_bathroom          0
Furniture               0
AC                      0
Parking                 0
house_type              0
floors_amount           0
flat_floor              0
Price                   0
Utilities               0
Rent_length             0
Prepay                  0
Pass_elevator           0
Cargo_elevator          0
windows                 0
ceiling_height          0
Garbage_disposal        0
maintenance             0
dtype: int64

In [241]:
df

,ID объявления,Rooms,Subway,Not_Moscow,New_address,time_to_subway,time_to_subway_car,time_to_subway_total,total_square,kitchen_square,...,Price,Utilities,Rent_length,Prepay,Pass_elevator,Cargo_elevator,windows,ceiling_height,Garbage_disposal,maintenance
0,271271157,4,Смоленская,0,"Москва, улица Новый Арбат, 27",9,0,9,200,20.0,...,500000.0,True,True,True,4,1,Во двор,3.00,Да,Дизайнерский
1,271634126,4,Смоленская,0,"Москва, улица Новый Арбат, 27",8,0,8,198,18.0,...,500000.0,True,True,True,1,1,На улицу и двор,3.50,Нет,Дизайнерский
2,271173086,4,Смоленская,0,"Москва, улица Новый Арбат, 27",7,0,7,200,4.0,...,500000.0,False,True,True,1,0,На улицу и двор,3.20,Нет,Евроремонт
3,272197456,4,Смоленская,0,"Москва, переулок Плотников, 21С1",3,0,3,170,17.0,...,400000.0,False,True,True,1,0,На улицу и двор,3.20,Нет,Евроремонт
4,273614615,2,Арбатская,0,"Москва, улица Новый Арбат, 15",7,0,7,58,5.0,...,225000.0,True,True,True,1,1,На улицу и двор,3.90,Да,Евроремонт
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,0,Говорово,1,"Москва, Боровское шоссе, 2к7,",8,0,8,35,8.0,...,42000.0,True,True,True,1,1,Во двор,3.00,Нет,Евроремонт
23364,274654844,1,Солнцево,1,"Москва, Производственная улица, 8к1,",7,0,7,38,11.0,...,45000.0,True,True,True,1,1,Во двор,3.00,Нет,Евроремонт
23365,268679909,2,Солнцево,0,"Москва, Боровский проезд, 11",6,0,6,43,43.1,...,50000.0,True,True,True,0,0,На улицу и двор,3.00,Нет,Дизайнерский
23366,274807525,2,Солнцево,1,"Москва, улица Богданова, 6к1,",11,0,11,52,10.0,...,55000.0,True,True,True,3,0,Во двор,2.65,Да,Евроремонт


In [242]:

df.rename(columns={'ID  объявления': 'ID'}, inplace=True)


In [243]:
df.to_csv('data.csv', index=False)